In [1]:
# 📦 Install dependencies (if running on Colab or fresh environment)
!pip install -q transformers datasets librosa jiwer


In [ ]:
# ✅ Load Whisper model and processor
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

model_checkpoint = "openai/whisper-small"  #244M params
processor = WhisperProcessor.from_pretrained(model_checkpoint)
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="fr", task="transcribe")
model.config.suppress_tokens = []
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [3]:
!pip install torchcodec

In [ ]:
!pip install -q datasets transformers accelerate torchaudio jiwer evaluate "fsspec<=2023.6.0"

from huggingface_hub import login
login(token="hf_cYYD***********************")  # Replace with your actual token

# ══ Load and clean dataset (Darija in Latin script)
from datasets import load_dataset

# Replace with your actual dataset
ds = load_dataset("atlasia/DODa-audio-dataset", download_mode="force_redownload")   #Contain small audios   #12hours of Darija audios



data/train-00000-of-00005.parquet:   0%|          | 0.00/333M [00:00<?, ?B/s]

data/train-00001-of-00005.parquet:   0%|          | 0.00/279M [00:00<?, ?B/s]

data/train-00002-of-00005.parquet:   0%|          | 0.00/237M [00:00<?, ?B/s]

data/train-00003-of-00005.parquet:   0%|          | 0.00/226M [00:00<?, ?B/s]

data/train-00004-of-00005.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12743 [00:00<?, ? examples/s]

In [5]:
from datasets import Audio

# 1. Force decoding
ds = ds.cast_column("audio", Audio(sampling_rate=16000))

# 2. Define validation
def is_valid(example):
    if not isinstance(example["darija_Latn"], str) or not example["darija_Latn"].strip():
        return False

    audio = example["audio"]

    # audio is now a dict
    if audio["array"] is None:
        return False
    if len(audio["array"]) == 0:
        return False

    return True

# 3. Filter dataset
ds_cleaned = ds["train"].filter(is_valid)

print("Cleaned size:", len(ds_cleaned))


Filter:   0%|          | 0/12743 [00:00<?, ? examples/s]

Cleaned size: 12743


In [ ]:
print(ds)
print(ds["train"].column_names)
print(ds["train"][0])


DatasetDict({
    train: Dataset({
        features: ['audio', 'darija_Latn', 'darija_Arab_new', 'english', 'darija_Arab_old'],
        num_rows: 12743
    })
})
['audio', 'darija_Latn', 'darija_Arab_new', 'english', 'darija_Arab_old']
{'audio': <datasets.features._torchcodec.AudioDecoder object at 0x7f82148f2360>, 'darija_Latn': 'homa mkhbbyin chi haja, ana mti99en!', 'darija_Arab_new': 'هوما مخبيين شي حاجة انا متيقن', 'english': "They're hiding something, I'm sure!", 'darija_Arab_old': 'هوما مخبّيين شي حاجة, أنا متيقّن!'}


In [ ]:
#  Prepare a single sample for inference
import random
import torch
example = ds_cleaned[0]
input_features = processor(example["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features

generated_ids = model.generate(
    input_features.to(model.device),
    max_length=128,
    no_repeat_ngram_size=3,
    #  remove forced_decoder_ids OR try "en" or "es"
    forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
)

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("🔊 Audio transcription:", transcription)
print("📝 Reference:", example["darija_Latn"])


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🔊 Audio transcription:  هم مخبنشي حاجة أنا متقن
📝 Reference: homa mkhbbyin chi haja, ana mti99en!


In [8]:
# 1. Préparation du dataset pour l'entraînement
def prepare_dataset(example):
    audio = example["audio"]
    # Extraire les log-mel spectrogrammes (features) du son
    example["input_features"] = processor.feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    # Tokenize la transcription
    example["labels"] = processor.tokenizer(
        example["darija_Latn"],
        truncation=True,
        max_length=225,
        return_tensors=None
    ).input_ids
    return example

ds_preprocessed = ds_cleaned.map(prepare_dataset, remove_columns=ds_cleaned.column_names)


# 2. Définir un DataCollator compatible Whisper
class WhisperDataCollator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        # On prépare input_features et labels séparément
        input_features = [{"input_features": f["input_features"]} for f in features]
        labels = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_batch = self.processor.tokenizer.pad(labels, return_tensors="pt")
        # Remplace le padding dans labels par -100 pour ignorer dans la loss
        batch["labels"] = label_batch["input_ids"].masked_fill(label_batch.attention_mask.ne(1), -100)
        return batch

data_collator = WhisperDataCollator(processor)


Map:   0%|          | 0/12743 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, Seq2SeqTrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from transformers import WhisperForConditionalGeneration
import types

# Charger modèle et config LoRA
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,   #rank
    lora_alpha=32,                      #scale
    lora_dropout=0.1,
    bias="none",
    target_modules=["k_proj", "v_proj"]  #based on the paper ("LoRA: Low-Rank Adaptation of Large Language Models")
)
model = get_peft_model(base_model, peft_config)

# Patcher base_model.forward pour supprimer input_ids et inputs_embeds
base_forward = model.base_model.forward

def base_model_forward_patch(self, *args, **kwargs):
    for arg in ["input_ids", "inputs_embeds"]:
        if arg in kwargs:
            kwargs.pop(arg)
    return base_forward(*args, **kwargs)

model.base_model.forward = types.MethodType(base_model_forward_patch, model.base_model)




In [ ]:
# Arguments d'entraînement (exemple)
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-darja-lora",
    per_device_train_batch_size=2,  # Batch size per device   / 2 audios processed per GPU per step.
    gradient_accumulation_steps=4, # Accumulate gradients over 4 steps
    learning_rate=1e-4,
    warmup_steps=100,    #For the first 100 steps, LR increases gradually from 0 → 1e-4.
    max_steps=500,    #instead of epochs
    save_steps=100,    #Every 100 steps, a checkpoint is saved
    eval_steps=100,
    logging_steps=50,
    report_to="none",   #No report to WandB, HF hub, MLflow, etc
    fp16=True,
    predict_with_generate=True,
    generation_max_length=225,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
)

# Créer Trainer (avec ton ds_preprocessed et data_collator définis)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_preprocessed,
    data_collator=data_collator,
)

# Lancer entraînement
trainer.train()

You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,7.362500
100,4.450000
150,3.058600
200,2.678000
250,2.440800
300,2.128300
350,1.622800
400,1.489000
450,1.591300
500,1.547000


TrainOutput(global_step=500, training_loss=2.8368255615234377, metrics={'train_runtime': 944.0067, 'train_samples_per_second': 4.237, 'train_steps_per_second': 0.53, 'total_flos': 1.15943768064e+18, 'train_loss': 2.8368255615234377, 'epoch': 0.31387319522912743})

In [11]:
# Dossier de sortie
output_dir = "./whisper-darja-lora"

# 1. Sauvegarder le modèle LoRA
model.save_pretrained(output_dir)

# 2. Sauvegarder le processor (tokenizer + feature extractor)
processor.save_pretrained(output_dir)

print(f"✅ Modèle et processor sauvegardés dans : {output_dir}")


✅ Modèle et processor sauvegardés dans : ./whisper-darja-lora


In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from peft import PeftModel, PeftConfig

# 1. Charger le processor
processor = WhisperProcessor.from_pretrained("./whisper-darja-lora")

# 2. Charger le modèle de base
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# 3. Appliquer les poids LoRA sur le modèle de base
model = PeftModel.from_pretrained(base_model, "./whisper-darja-lora")

# 4. Passer en eval mode
model.eval()


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 768)
          (layers): ModuleList(
            (0-11): 12 x WhisperEncoderLayer(
              (self_attn): WhisperAttention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=768, 

**TEST SUR UNE NOUVELLE DATA SET**

In [14]:
from datasets import load_dataset

ds = load_dataset("Snousnou/Moroccan-Darija-ASR")
example = ds["train"][2]
print(example.keys())


dict_keys(['file', 'text'])


In [ ]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
import torch

# Charger dataset
ds = load_dataset("Snousnou/Moroccan-Darija-ASR")
example = ds["train"][2]

print("Clés disponibles :", example.keys())
print("Référence :", example["text"])

# Récupérer audio (array + sample_rate)
audio_array = example["file"]["array"]
sample_rate = example["file"]["sampling_rate"]

# Charger modèle
processor = WhisperProcessor.from_pretrained("./whisper-darja-lora")
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model = PeftModel.from_pretrained(base_model, "./whisper-darja-lora")   # we applique LoRA adapters sauvegardés
model.eval()

# Préparer features / transforme l’audio en input features (mel spectrograms) que Whisper peut utiliser
inputs = processor(audio_array, sampling_rate=sample_rate, return_tensors="pt").input_features.to(model.device)

# Générer transcription
with torch.no_grad():
   generated_ids = model.base_model.generate(
    inputs,
    max_length=128,
    forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
)


transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("Prédiction :", transcription)


Clés disponibles : dict_keys(['file', 'text'])
Référence : bghit nkhles dariba
Prédiction : git n7las dtariabha


In [16]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

# Charger dataset et un exemple
ds = load_dataset("Snousnou/Moroccan-Darija-ASR")
example = ds["train"][2]
audio_array = example["file"]["array"]
sample_rate = example["file"]["sampling_rate"]

# Charger processor et modèle de base Whisper-small
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
base_model.eval()

# Préparer les features
inputs = processor(audio_array, sampling_rate=sample_rate, return_tensors="pt").input_features

# Générer transcription avec Whisper-small (base)
with torch.no_grad():
    generated_ids_base = base_model.generate(
        inputs,
        max_length=128,
        forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
    )
transcription_base = processor.batch_decode(generated_ids_base, skip_special_tokens=True)[0]

print("📝 Référence :", example["text"])
print("🤖 Whisper-small base :", transcription_base)


📝 Référence : bghit nkhles dariba
🤖 Whisper-small base :  بريبن خلص بطاريب


In [17]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
import torch
from jiwer import wer

# Charger dataset (tu peux changer 'train' en 'test' si tu préfères)
ds = load_dataset("Snousnou/Moroccan-Darija-ASR", split="train")

# Charger processor et modèles
processor = WhisperProcessor.from_pretrained("./whisper-darja-lora")
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
base_model.eval()

base_model.to("cuda" if torch.cuda.is_available() else "cpu")

# Charger modèle LoRA
base_model_for_lora = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model_lora = PeftModel.from_pretrained(base_model_for_lora, "./whisper-darja-lora")
model_lora.eval()
model_lora.to("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Limite le nombre d'exemples pour test rapide
max_examples = 10

wers_base = []
wers_lora = []

for i, example in enumerate(ds):
    if i >= max_examples:
        break

    audio = example["file"]["array"]
    sample_rate = example["file"]["sampling_rate"]
    reference = example["text"]

    inputs = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_features.to(device)

    # Base model generate
    with torch.no_grad():
        generated_ids_base = base_model.generate(
            inputs,
            max_length=128,
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
        )
    pred_base = processor.batch_decode(generated_ids_base, skip_special_tokens=True)[0]

    # LoRA model generate (attention à generate sur base_model)
    with torch.no_grad():
        generated_ids_lora = model_lora.base_model.generate(
            inputs,
            max_length=128,
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
        )
    pred_lora = processor.batch_decode(generated_ids_lora, skip_special_tokens=True)[0]

    # Calcul WER (tu peux normaliser les textes si besoin)
    wer_base = wer(reference, pred_base)
    wer_lora = wer(reference, pred_lora)

    wers_base.append(wer_base)
    wers_lora.append(wer_lora)

    print(f"\nExemple {i+1}:")
    print("Référence     :", reference)
    print("Prédiction Base :", pred_base)
    print(f"WER Base      : {wer_base:.3f}")
    print("Prédiction LoRA :", pred_lora)
    print(f"WER LoRA      : {wer_lora:.3f}")

print("\n=== Résultats moyens ===")
print(f"WER moyen Base : {sum(wers_base)/len(wers_base):.3f}")
print(f"WER moyen LoRA : {sum(wers_lora)/len(wers_lora):.3f}")



Exemple 1:
Référence     : bghit nsift lflous
Prédiction Base :  ونحن نقوم بعمل سفت الفلوس
WER Base      : 1.667
Prédiction LoRA : gripp nsafet lflouz
WER LoRA      : 1.000

Exemple 2:
Référence     : ch7al f solde
Prédiction Base :  شهل في السول
WER Base      : 1.000
Prédiction LoRA : chhl fSoul?
WER LoRA      : 1.000

Exemple 3:
Référence     : bghit nkhles dariba
Prédiction Base :  بريبن خلص بطاريب
WER Base      : 1.000
Prédiction LoRA : git n7las dtariabha
WER LoRA      : 1.000

Exemple 4:
Référence     : ch7al flcompte
Prédiction Base :  شهل عندي في الكنت
WER Base      : 2.000
Prédiction LoRA : ch7el3ndi f lkont
WER LoRA      : 1.500

Exemple 5:
Référence     : bghit nkhrej lflous
Prédiction Base :  ريتم خرزر فلو
WER Base      : 1.000
Prédiction LoRA : ghit n9arraj
WER LoRA      : 1.000

Exemple 6:
Référence     : bghit nkhles lma o do
Prédiction Base :  بريد نخلص
WER Base      : 1.000
Prédiction LoRA : bghit n9llas
WER LoRA      : 0.800

=== Résultats moyens ===
WER moyen Base :

In [18]:
import shutil

shutil.make_archive("whisper-darja-lora", 'zip', output_dir)
print("Archive whisper-darja-lora.zip créée.")


Archive whisper-darja-lora.zip créée.


**TEST SUR : atlasia/DODa-audio-dataset**

In [19]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
import torch
from jiwer import wer
import types

ds = load_dataset("atlasia/DODa-audio-dataset", split="train")

processor = WhisperProcessor.from_pretrained("./whisper-darja-lora")

base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
base_model.eval()
base_model.to("cuda" if torch.cuda.is_available() else "cpu")

base_model_for_lora = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model_lora = PeftModel.from_pretrained(base_model_for_lora, "./whisper-darja-lora")
model_lora.eval()
model_lora.to("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Patch nécessaire sur forward
base_forward = model_lora.base_model.forward
def base_model_forward_patch(self, *args, **kwargs):
    for arg in ["input_ids", "inputs_embeds"]:
        if arg in kwargs:
            kwargs.pop(arg)
    return base_forward(*args, **kwargs)
model_lora.base_model.forward = types.MethodType(base_model_forward_patch, model_lora.base_model)

max_examples = 10

wers_base = []
wers_lora = []

for i, example in enumerate(ds):
    if i >= max_examples:
        break

    audio = example["audio"]["array"]
    sample_rate = example["audio"]["sampling_rate"]
    reference = example["darija_Latn"]

    inputs = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_features.to(device)

    with torch.no_grad():
        generated_ids_base = base_model.generate(
            inputs,
            max_length=128,
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
        )
    pred_base = processor.batch_decode(generated_ids_base, skip_special_tokens=True)[0]

    with torch.no_grad():
        generated_ids_lora = model_lora.base_model.generate(
            input_features=inputs,
            max_length=128,
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
        )
    pred_lora = processor.batch_decode(generated_ids_lora, skip_special_tokens=True)[0]

    wer_base = wer(reference, pred_base)
    wer_lora = wer(reference, pred_lora)

    wers_base.append(wer_base)
    wers_lora.append(wer_lora)

    print(f"\nExemple {i+1}:")
    print("Référence     :", reference)
    print("Prédiction Base :", pred_base)
    print(f"WER Base      : {wer_base:.3f}")
    print("Prédiction LoRA :", pred_lora)
    print(f"WER LoRA      : {wer_lora:.3f}")

print("\n=== Résultats moyens ===")
print(f"WER moyen Base : {sum(wers_base)/len(wers_base):.3f}")
print(f"WER moyen LoRA : {sum(wers_lora)/len(wers_lora):.3f}")



Exemple 1:
Référence     : homa mkhbbyin chi haja, ana mti99en!
Prédiction Base :  هم مخبنشي حاجة أنا متقن
WER Base      : 1.000
Prédiction LoRA : ho ma mkhbin chi 3aja anam ti9an
WER LoRA      : 1.000

Exemple 2:
Référence     : bayna homa tay7awlo ib9aw mbrrdin.
Prédiction Base :  بينهما كحولوا بقام بردين
WER Base      : 1.000
Prédiction LoRA : baina homa ki7awlo b9am brdin
WER LoRA      : 0.800

Exemple 3:
Référence     : loTilat mabaynach fihom mori7in bzzaf.
Prédiction Base :  لو طيلات ما بيناش فيهم مورحين بالزيف
WER Base      : 1.400
Prédiction LoRA : lotaylat mbeynach fiyahom mori7in bzaf
WER LoRA      : 0.800

Exemple 4:
Référence     : ghaliban ghayjrriw 3lih mn lkhdma!
Prédiction Base :  غالباً غيروا عليهم الخدمة
WER Base      : 1.000
Prédiction LoRA : ghali b9ayjriwa 3lih ml khdma
WER LoRA      : 0.800

Exemple 5:
Référence     : Tab3an rah mkta2eb!
Prédiction Base :  طبعا لنرحمك تائب
WER Base      : 1.000
Prédiction LoRA : tab3ana rahma ktayib
WER LoRA      : 1.000

Exempl

Interpretation !?

Spelling Ambiguity: "Arabizi" has no standard spelling (e.g., "kifach", "kifech", "kfach"). The model struggles to guess the exact spelling preference of the labeler.

Detailed Error Analysis !! ❎


The fine-tuning has successfully achieved its primary goal: Script Switching.

Base Model: Consistently outputs Arabic script.

LoRA Model: Consistently outputs Latin script (Arabizi).

However, the Word Error Rate (WER) remains high (~1.103) primarily due to segmentation issues (spacing) and the lack of standardized spelling in Darija, rather than a failure to understand the audio content.


***1. The "Space" & Segmentation Problem***

The most significant source of error in the LoRA model is the incorrect splitting or merging of words. Because Darija in Latin script is not standardized, the model struggles to decide where one word ends and another begins.

Example 1:

Reference: homa (they)

LoRA: ho ma

Impact: This counts as 1 substitution and 1 insertion, heavily penalizing the WER, even though the pronunciation is identical.

***2. Synonyms and Dialect Variations***

The model sometimes substitutes the specific dialect word used in the reference with a common synonym.

Example 2 & 6 (The "Tay/Ka" Shift):

Reference: Uses the prefix tay (e.g., tay7awlo, tayballia) to indicate present continuous tense.

LoRA: Swaps it for ki or kay (e.g., ki7awlo, kaybal).



***3. Spelling Inconsistencies (Arabizi Normalization)***

Since Arabizi has no official orthography, the model and the reference often disagree on how to spell a sound.

Example 3:

Reference: bzzaf (a lot)

LoRA: bzaf

Insight: Double consonants (zz) are often reduced to single consonants (z) by the model.

**To Double CHECK , we use the seconde metric : Caracter error rate**

---



In [22]:
from jiwer import cer  # Add cer import at the top

# In your evaluation loop, add CER lists
wers_base = []
wers_lora = []
cers_base = []  # NEW
cers_lora = []  # NEW

for i, example in enumerate(ds):
    if i >= max_examples:
        break

    audio = example["audio"]["array"]
    sample_rate = example["audio"]["sampling_rate"]
    reference = example["darija_Latn"]

    inputs = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_features.to(device)

    # Base model prediction
    with torch.no_grad():
        generated_ids_base = base_model.generate(
            inputs,
            max_length=128,
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
        )
    pred_base = processor.batch_decode(generated_ids_base, skip_special_tokens=True)[0]

    # LoRA model prediction
    with torch.no_grad():
        generated_ids_lora = model_lora.base_model.generate(
            input_features=inputs,
            max_length=128,
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
        )
    pred_lora = processor.batch_decode(generated_ids_lora, skip_special_tokens=True)[0]

    # Calculate WER (existing)
    wer_base = wer(reference, pred_base)
    wer_lora = wer(reference, pred_lora)

    # Calculate CER (NEW)
    cer_base = cer(reference, pred_base)
    cer_lora = cer(reference, pred_lora)

    wers_base.append(wer_base)
    wers_lora.append(wer_lora)
    cers_base.append(cer_base)  # NEW
    cers_lora.append(cer_lora)  # NEW

    print(f"\nExemple {i+1}:")
    print("Référence     :", reference)
    print("Prédiction Base :", pred_base)
    print(f"WER Base      : {wer_base:.3f}")
    print(f"CER Base      : {cer_base:.3f}")  # NEW
    print("Prédiction LoRA :", pred_lora)
    print(f"WER LoRA      : {wer_lora:.3f}")
    print(f"CER LoRA      : {cer_lora:.3f}")  # NEW

print("\n=== Résultats moyens ===")
print(f"WER moyen Base : {sum(wers_base)/len(wers_base):.3f}")
print(f"WER moyen LoRA : {sum(wers_lora)/len(wers_lora):.3f}")
print(f"CER moyen Base : {sum(cers_base)/len(cers_base):.3f}")  # NEW
print(f"CER moyen LoRA : {sum(cers_lora)/len(cers_lora):.3f}")  # NEW


Exemple 1:
Référence     : homa mkhbbyin chi haja, ana mti99en!
Prédiction Base :  هم مخبنشي حاجة أنا متقن
WER Base      : 1.000
CER Base      : 0.889
Prédiction LoRA : ho ma mkhbin chi 3aja anam ti9an
WER LoRA      : 1.000
CER LoRA      : 0.278

Exemple 2:
Référence     : bayna homa tay7awlo ib9aw mbrrdin.
Prédiction Base :  بينهما كحولوا بقام بردين
WER Base      : 1.000
CER Base      : 0.912
Prédiction LoRA : baina homa ki7awlo b9am brdin
WER LoRA      : 0.800
CER LoRA      : 0.265

Exemple 3:
Référence     : loTilat mabaynach fihom mori7in bzzaf.
Prédiction Base :  لو طيلات ما بيناش فيهم مورحين بالزيف
WER Base      : 1.400
CER Base      : 0.921
Prédiction LoRA : lotaylat mbeynach fiyahom mori7in bzaf
WER LoRA      : 0.800
CER LoRA      : 0.237

Exemple 4:
Référence     : ghaliban ghayjrriw 3lih mn lkhdma!
Prédiction Base :  غالباً غيروا عليهم الخدمة
WER Base      : 1.000
CER Base      : 0.912
Prédiction LoRA : ghali b9ayjriwa 3lih ml khdma
WER LoRA      : 0.800
CER LoRA      : 0.29